In [1]:
from itertools import combinations, permutations
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from rules import divisiveness, win_rate, bootstrap3, bootstrap4, copeland, divisiveness_copeland
import math

In [2]:
def imperfect_population(n_individuals, all_rankings):
    # Impartial culture with repetition
    ids = 0
    output = []
    for times in range(n_individuals):
        df_tmp = pd.DataFrame(list(combinations(all_rankings[times % len(all_rankings)], 2)), \
                              columns=["option_a", "option_b"])
        df_tmp["uuid"] = ids
        output.append(df_tmp)
        ids = ids + 1
        all_rankings = shuffle(all_rankings)
    return output
        
def perfect_population(n_times, all_rankings):
    # Impartial culture
    ids = 0
    output = []
    for times in range(n_times):
        for each_ranking in all_rankings:
            df_tmp = pd.DataFrame(list(combinations(each_ranking, 2)), \
                                  columns=["option_a", "option_b"])
            df_tmp["uuid"] = ids
            output.append(df_tmp)
            ids = ids + 1
    return output


def data_transform(all_rankings):
        # Impartial culture
    ids = 0
    output = []
    for each_ranking in all_rankings:
        df_tmp = pd.DataFrame(list(combinations(each_ranking, 2)), \
                              columns=["option_a", "option_b"])
        df_tmp["uuid"] = ids
        output.append(df_tmp)
        ids = ids + 1
    return output
    

def using_preflib(number_proposals, number_individuals,type_='IC'):
    from preflibtools.instances import OrdinalInstance
    instance = OrdinalInstance()
#     5 voters and 10 alternatives
    # instance.populate_mallows_mix(5, 10, 3)
    # instance.populate_urn(5, 10, 76)
    # instance.populate_IC(5, 10)
    # instance.populate_IC_anon(5, 10)
    from preflibtools.properties import borda_scores, has_condorcet
    if type_ == 'IC':
        instance.populate_IC(number_individuals, number_proposals)
    elif type_ == 'UM10':
        instance.populate_urn(number_individuals, number_proposals, (math.factorial(number_proposals)/9))
    elif type_ == 'UM50':
        instance.populate_urn(number_individuals, number_proposals, (math.factorial(number_proposals)))
    return list([[y[0] for y in x] for x in instance.full_profile()])

def standard_deviation_rankings(all_rankings):
#     np.array([[10,1],[5,2],[1,3]])[:,1])
    aux1 = pd.DataFrame(all_rankings).melt().groupby('value')['variable'].std().reset_index()
    aux1.columns = ['id','value']
    aux1['rank'] = aux1['value'].rank(method='average', ascending=False)
    return aux1

In [3]:
standard_deviation_rankings([[2,1,0],[0,1,2]])

,id,value,rank
0,0,1.414214,1.5
1,1,0.000000,3.0
2,2,1.414214,1.5


In [4]:
aux1 = pd.DataFrame([[2,1,0],[0,1,2]]).melt().groupby('value')['variable'].std().reset_index()
aux1.columns = ['id','value']
aux1['rank'] = aux1['value'].rank(method='average', ascending=False)
aux1

,id,value,rank
0,0,1.414214,1.5
1,1,0.000000,3.0
2,2,1.414214,1.5


In [5]:
from preflibtools.instances import OrdinalInstance
from preflibtools.properties import borda_scores, has_condorcet
instance = OrdinalInstance()
number_proposals = 18
number_individuals = 500
# instance.populate_IC(number_individuals, number_proposals)
instance.populate_urn(number_individuals, number_proposals, (math.factorial(number_proposals)))
len(list([[y[0] for y in x] for x in instance.full_profile()]))

500

In [6]:
# 20 proposals 25 individuals
# 19 proposals 100 individuals
# 18 proposals 500 individuals

In [ ]:
data_type = 'UM10'
N_ITERATION = 1000
import warnings; warnings.simplefilter('ignore')
results = []

for simulation in range(N_ITERATION):
    
    print("simulation", simulation)
    
    for number_proposals in range(3,19):

        for number_individuals in [100, 200, 300, 400, 500]:
            
#             print("started...", simulation)
    #         all_rankings = shuffle(list(permutations(range(1, number_proposals+1), number_proposals)))
            all_rankings = using_preflib(number_proposals, number_individuals, type_=data_type)
            output = data_transform(all_rankings)

#             print("population...")
            data = pd.concat(output).reset_index(drop=True)
            data["id"] = range(1, data.shape[0] + 1)
            data["selected"] = data["option_a"]
            data["option_a_sorted"] = data[["option_a", "option_b"]].min(axis=1).astype(int)
            data["option_b_sorted"] = data[["option_a", "option_b"]].max(axis=1).astype(int)
            data["card_id"] = data["option_a_sorted"].astype(str) + "_" + data["option_b_sorted"].astype(str)

            a = data[["option_a", "option_b", "selected"]].values
            data["option_source"] = np.where(a[:, 1] == a[:, 2], a[:, 0], a[:, 1])
            data["option_target"] = np.where(a[:, 0] == a[:, 2], a[:, 0], a[:, 1])
            
#             print("win rate...")
            df_win_rate = bootstrap4(data, win_rate, iterations=1, 
                                     aggregate=False, rank=True, rank_column="rank")
    
            df_copeland = bootstrap4(data, copeland, iterations=1, 
                                     aggregate=False, rank=True, rank_column="rank")

    #         df_win_rate.to_csv("IJCAI/winrate_%s.csv"%number_proposals, index=False)
#             print("divisiveness...")
            df_divisiveness = bootstrap4(data, divisiveness, iterations=1, 
                                         aggregate=False, rank=True, rank_column="rank")
    
    
            df_divisiveness_copeland = bootstrap4(data, divisiveness_copeland, iterations=1, 
                                         aggregate=False, rank=True, rank_column="rank")
    #         df_divisiveness.to_csv("IJCAI/divisiveness_%s.csv"%number_proposals, index=False)
    
#             print("df_std...")
            df_std = standard_deviation_rankings(all_rankings)
            #         df_std.to_csv("IJCAI/std_%s.csv"%number_proposals, index=False)
            del all_rankings 
            results.append([number_proposals, 
                            number_individuals,
                            simulation,
                    list(df_win_rate.sort_values(by='value', ascending=False)['id'].values),\
                    list(df_copeland.sort_values(by='value', ascending=False)['id'].values),\
                    list(df_std.sort_values(by='value', ascending=False)['id'].values),\
                            
                    list(df_divisiveness.sort_values(by='value', ascending=False)['id'].values),\
                   list(df_divisiveness_copeland.sort_values(by='value', ascending=False)['id'].values)])

    plot = pd.DataFrame(results, columns=['number_proposals','number_individuals','simulation',\
                                          'RANK(WR)','RANK(COP)',
                                            'RANK(STD)','RANK(DIV(WR))','RANK(DIV(COP))'])
    plot.to_csv("IJCAI/results_kt_big_%s_%s_p2.csv"%(data_type,N_ITERATION), index=False)


simulation 0


  0%|          | 0/6 [00:00<?, ?it/s]

simulation 1


  0%|          | 0/6 [00:00<?, ?it/s]

simulation 2


...

100%|██████████| 6/6 [00:00<00:00, 88.09it/s]

simulation 19



100%|██████████| 6/6 [00:00<00:00, 88.25it/s]

simulation 20



 23%|██▎       | 63/272 [00:02<00:08, 24.14it/s]

In [ ]:
plot.head(10)

In [ ]:
#Figure1: kendall-tau on the vertical axis, 
# number of individuals on horiz axis (fix number of items=20)

#Figure1: kendall-tau on the vertical axis, 
# number of individuals on horiz axis (fix number of items=20)
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2, figsize=(8,4), dpi=100)
sns.lineplot(data=plot[(plot['number_individuals']==plot['number_individuals'].max())], \
             x='number_proposals', y='KT(STD,DIV)', 
             linewidth=3, alpha=1, color
             ='darkred', ax=ax[0])
sns.lineplot(data=plot[(plot['number_individuals']==plot['number_individuals'].max())], \
             x='number_proposals', y='KT(WR,DIV)', 
             linewidth=3, alpha=1, color
             ='darkblue', ax=ax[1])

# Hide the right and top spines
# change all spines
for i in range(2):
    ax[i].spines["left"].set_linewidth(1.2)
    ax[i].spines["bottom"].set_linewidth(1.2)
    ax[i].spines.right.set_visible(False)
    ax[i].spines.top.set_visible(False)
    ax[i].set_ylim(-1.05,1.05)

plt.tight_layout()
# plt.legend(title='',bbox_to_anchor=(1.2,0.9))

In [ ]:
#Figure1: kendall-tau on the vertical axis, 
# number of individuals on horiz axis (fix number of items=20)

#Figure1: kendall-tau on the vertical axis, 
# number of individuals on horiz axis (fix number of items=20)
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2, figsize=(8,4), dpi=100)
sns.lineplot(data=plot[(plot['number_individuals']==plot['number_individuals'].max())], \
             x='number_proposals', y='KT(STD,DIV)', 
             linewidth=3, alpha=1, color
             ='darkred', ax=ax[0])
sns.lineplot(data=plot[(plot['number_individuals']==plot['number_individuals'].max())], \
             x='number_proposals', y='KT(WR,DIV)', 
             linewidth=3, alpha=1, color
             ='darkblue', ax=ax[1])

# Hide the right and top spines
# change all spines
for i in range(2):
    ax[i].spines["left"].set_linewidth(1.2)
    ax[i].spines["bottom"].set_linewidth(1.2)
    ax[i].spines.right.set_visible(False)
    ax[i].spines.top.set_visible(False)
    ax[i].set_ylim(-1.05,1.05)

plt.tight_layout()
# plt.legend(title='',bbox_to_anchor=(1.2,0.9))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(dpi=100)
ax = sns.heatmap(data=plot.pivot_table(index='number_proposals',\
                                       columns='number_individuals', values='KT(WR,DIV)', aggfunc='mean'), 
             linewidth=0, alpha=0.9, cmap='RdBu', vmin=-0.5, vmax=0.5, 
                             cbar_kws={'label': 'KT(WR,DIV)'})
ax.invert_yaxis()
plt.title('%s'%data_type)
# plt.legend(title='',bbox_to_anchor=(1.05,0.9))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(dpi=100)
ax = sns.heatmap(data=plot.pivot_table(index='number_proposals',\
                                       columns='number_individuals', values='KT(STD,DIV)', aggfunc='mean'), 
             linewidth=0, alpha=0.9, cmap='RdBu', vmin=-1, vmax=1, 
                             cbar_kws={'label': 'KT(STD,DIV)'})
ax.invert_yaxis()
plt.title('%s'%data_type)
# plt.legend(title='',bbox_to_anchor=(1.05,0.9))


In [ ]:
plot.head(1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(dpi=100)
sns.lineplot(data=plot[plot['number_individuals']==100], \
             x='number_proposals', y='KT(STD,DIV)', hue='number_individuals', 
             linewidth=3, alpha=1, palette
             ='Reds_r')
sns.lineplot(data=plot[plot['number_individuals']==100], \
             x='number_proposals', y='KT(WR,DIV)', hue='number_individuals', 
             linewidth=3, alpha=1, palette
             ='Blues_r')
plt.legend(title='',bbox_to_anchor=(1.2,0.9))

In [ ]:
plot2 = plot[['number_proposals','number_individuals',\
             'KT(WR,DIV)','KT(WR,STD)','KT(STD,DIV)']]\
.melt(id_vars=['number_proposals','number_individuals'])
plot2.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(dpi=100)
sns.lineplot(data=plot2, x='number_proposals', y='value', hue='number_individuals', 
             linewidth=3, alpha=0.9, style='variable', palette='inferno_r')